In [2703]:
# Amy Noyes
# BIOL 51000: Data Systems Life Sciences
# Week 5 Coding Assignment: Calculating Root-Mean-Square Deviation (RMSD) and aligning a Structure Ensemble

In [2704]:
# import math
# import Bio
from Bio import PDB
from math import pi, sqrt, degrees, acos
from matplotlib import pyplot
from numpy import array, dot, zeros, ones, cross, sqrt, linalg, exp, identity  

In [2705]:
# defines 1 letter amino acid codes
amino_acids = {'ALA':'A','CYS':'C','ASP':'D','GLU':'E',
                       'PHE':'F','GLY':'G','HIS':'H','ILE':'I',
                       'LYS':'K','LEU':'L','MET':'M','ASN':'N',
                       'PRO':'P','GLN':'Q','ARG':'R','SER':'S',
                       'THR':'T','VAL':'V','TRP':'W','TYR':'Y',
                       'G':'G','C':'C','A':'A','T':'T','U':'U'}

In [2706]:
# atomic numbers commonly found in biological molecules
atomic_numbers = {'H':1, 'C':12, 'N':14, 'O':16, 'P':31, 'S':32}

In [2707]:
# substitution matrix for protein sequence

BLOSUM62 = {'A':{'A': 4,'R':-1,'N':-2,'D':-2,'C': 0,'Q':-1,'E':-1,'G': 0,'H':-2,'I':-1,
                 'L':-1,'K':-1,'M':-1,'F':-2,'P':-1,'S': 1,'T': 0,'W':-3,'Y':-2,'V': 0,'X':0},
            'R':{'A':-1,'R': 5,'N': 0,'D':-2,'C':-3,'Q': 1,'E': 0,'G':-2,'H': 0,'I':-3,
                 'L':-2,'K': 2,'M':-1,'F':-3,'P':-2,'S':-1,'T':-1,'W':-3,'Y':-2,'V':-3,'X':0},
            'N':{'A':-2,'R': 0,'N': 6,'D': 1,'C':-3,'Q': 0,'E': 0,'G': 0,'H': 1,'I':-3,
                 'L':-3,'K': 0,'M':-2,'F':-3,'P':-2,'S': 1,'T': 0,'W':-4,'Y':-2,'V':-3,'X':0},
            'D':{'A':-2,'R':-2,'N': 1,'D': 6,'C':-3,'Q': 0,'E': 2,'G':-1,'H':-1,'I':-3,
                 'L':-4,'K':-1,'M':-3,'F':-3,'P':-1,'S': 0,'T':-1,'W':-4,'Y':-3,'V':-3,'X':0},
            'C':{'A': 0,'R':-3,'N':-3,'D':-3,'C': 9,'Q':-3,'E':-4,'G':-3,'H':-3,'I':-1,
                 'L':-1,'K':-3,'M':-1,'F':-2,'P':-3,'S':-1,'T':-1,'W':-2,'Y':-2,'V':-1,'X':0},
            'Q':{'A':-1,'R': 1,'N': 0,'D': 0,'C':-3,'Q': 5,'E': 2,'G':-2,'H': 0,'I':-3,
                 'L':-2,'K': 1,'M': 0,'F':-3,'P':-1,'S': 0,'T':-1,'W':-2,'Y':-1,'V':-2,'X':0},
            'E':{'A':-1,'R': 0,'N': 0,'D': 2,'C':-4,'Q': 2,'E': 5,'G':-2,'H': 0,'I':-3,
                 'L':-3,'K': 1,'M':-2,'F':-3,'P':-1,'S': 0,'T':-1,'W':-3,'Y':-2,'V':-2,'X':0},
            'G':{'A': 0,'R':-2,'N': 0,'D':-1,'C':-3,'Q':-2,'E':-2,'G': 6,'H':-2,'I':-4,
                 'L':-4,'K':-2,'M':-3,'F':-3,'P':-2,'S': 0,'T':-2,'W':-2,'Y':-3,'V':-3,'X':0},
            'H':{'A':-2,'R': 0,'N': 1,'D':-1,'C':-3,'Q': 0,'E': 0,'G':-2,'H': 8,'I':-3,
                 'L':-3,'K':-1,'M':-2,'F':-1,'P':-2,'S':-1,'T':-2,'W':-2,'Y': 2,'V':-3,'X':0},
            'I':{'A':-1,'R':-3,'N':-3,'D':-3,'C':-1,'Q':-3,'E':-3,'G':-4,'H':-3,'I': 4,
                 'L': 2,'K':-3,'M': 1,'F': 0,'P':-3,'S':-2,'T':-1,'W':-3,'Y':-1,'V': 3,'X':0},
            'L':{'A':-1,'R':-2,'N':-3,'D':-4,'C':-1,'Q':-2,'E':-3,'G':-4,'H':-3,'I': 2,
                 'L': 4,'K':-2,'M': 2,'F': 0,'P':-3,'S':-2,'T':-1,'W':-2,'Y':-1,'V': 1,'X':0},
            'K':{'A':-1,'R': 2,'N': 0,'D':-1,'C':-3,'Q': 1,'E': 1,'G':-2,'H':-1,'I':-3,
                 'L':-2,'K': 5,'M':-1,'F':-3,'P':-1,'S': 0,'T':-1,'W':-3,'Y':-2,'V':-2,'X':0},
            'M':{'A':-1,'R':-1,'N':-2,'D':-3,'C':-1,'Q': 0,'E':-2,'G':-3,'H':-2,'I': 1,
                 'L': 2,'K':-1,'M': 5,'F': 0,'P':-2,'S':-1,'T':-1,'W':-1,'Y':-1,'V': 1,'X':0},
            'F':{'A':-2,'R':-3,'N':-3,'D':-3,'C':-2,'Q':-3,'E':-3,'G':-3,'H':-1,'I': 0,
                 'L': 0,'K':-3,'M': 0,'F': 6,'P':-4,'S':-2,'T':-2,'W': 1,'Y': 3,'V':-1,'X':0},
            'P':{'A':-1,'R':-2,'N':-2,'D':-1,'C':-3,'Q':-1,'E':-1,'G':-2,'H':-2,'I':-3,
                 'L':-3,'K':-1,'M':-2,'F':-4,'P': 7,'S':-1,'T':-1,'W':-4,'Y':-3,'V':-2,'X':0},
            'S':{'A': 1,'R':-1,'N': 1,'D': 0,'C':-1,'Q': 0,'E': 0,'G': 0,'H':-1,'I':-2,
                 'L':-2,'K': 0,'M':-1,'F':-2,'P':-1,'S': 4,'T': 1,'W':-3,'Y':-2,'V':-2,'X':0},
            'T':{'A': 0,'R':-1,'N': 0,'D':-1,'C':-1,'Q':-1,'E':-1,'G':-2,'H':-2,'I':-1,
                 'L':-1,'K':-1,'M':-1,'F':-2,'P':-1,'S': 1,'T': 5,'W':-2,'Y':-2,'V': 0,'X':0},
            'W':{'A':-3,'R':-3,'N':-4,'D':-4,'C':-2,'Q':-2,'E':-3,'G':-2,'H':-2,'I':-3,
                 'L':-2,'K':-3,'M':-1,'F': 1,'P':-4,'S':-3,'T':-2,'W':11,'Y': 2,'V':-3,'X':0},
            'Y':{'A':-2,'R':-2,'N':-2,'D':-3,'C':-2,'Q':-1,'E':-2,'G':-3,'H': 2,'I':-1,
                 'L':-1,'K':-2,'M':-1,'F': 3,'P':-3,'S':-2,'T':-2,'W': 2,'Y': 7,'V':-1,'X':0},
            'V':{'A': 0,'R':-3,'N':-3,'D':-3,'C':-1,'Q':-2,'E':-2,'G':-3,'H':-3,'I': 3,
                 'L': 1,'K':-2,'M': 1,'F':-1,'P':-2,'S':-2,'T': 0,'W':-3,'Y':-1,'V': 4,'X':0},
            'X':{'A': 0,'R': 0,'N': 0,'D': 0,'C': 0,'Q': 0,'E': 0,'G': 0,'H': 0,'I': 0,
                 'L': 0,'K': 0,'M': 0,'F': 0,'P': 0,'S': 0,'T': 0,'W': 0,'Y': 0,'V': 0,'X':0}}

In [2708]:
# substitution matrix for DNA
DNA_1 = {'G': { 'G': 1, 'C':-3, 'A':-3, 'T':-3, 'N':0 },
         'C': { 'G':-3, 'C': 1, 'A':-3, 'T':-3, 'N':0 },
         'A': { 'G':-3, 'C':-3, 'A': 1, 'T':-3, 'N':0 },
         'T': { 'G':-3, 'C':-3, 'A':-3, 'T': 1, 'N':0 },
         'N': { 'G': 0, 'C': 0, 'A': 0, 'T': 0, 'N':0 }}  

In [2709]:
# import Protein Data Bank files directly from PDB
# import urlopen function
try:
  # for python 3
  from urllib.request import urlopen
except ImportError:
  # for python 2
  from urllib2 import urlopen
    
# update PDB url as needed
PDB_URL = 'http://files.rcsb.org/view/' \
          '%s.pdb?format=PDB&compression=None'

In [2710]:
# download PDB file by ID and filename if available
def download_PDB(pdbId, fileName=None):

  # for no filename: create filename by appending .pdb to ID
  if not fileName:
    fileName = '%s.pdb' % pdbId

  # retrieve file
  retrieved = urlopen(PDB_URL % pdbId)
  # convert bytes to string using UTF-8  
  data = retrieved.read().decode('utf-8')

  # write download to file
  fileObj = open(fileName, 'w')
  fileObj.write(data)
  fileObj.close()

  return fileName

In [2711]:
# creates structure object from PDB file
def get_structures(fileName):

  fileObject = open(fileName)
  structure = None
  name = 'unknown'
  conformation = 0
  pdbId = None
  structures = []

  # loops through each line
  for line in fileObject:
    record = line[0:6].strip()
    
    if record == 'HEADER':
      pdbId = line.split()[-1]

    elif record == 'TITLE':
      name = line[10:].strip()

    elif record == 'MODEL':
      conformation = int(line[10:14])

    elif record == 'ENDMDL':
      structure = None

    elif record == 'ATOM':
      atomName   = line[12:16].strip()
      resName    = line[17:20].strip()
      chain_code  = line[21:22].strip()
      seqId      = int(line[22:26])
      x          = float(line[30:38])
      y          = float(line[38:46])
      z          = float(line[46:54])
      segment    = line[72:76].strip()
      element    = line[76:78].strip() 
      
      # create new chain  
      if chain_code == '':
        if segment:
          chain_code = segment
        else:
          chain_code = 'A'

      if not structure:
        structure = Structure(name, conformation, pdbId)
        structures.append(structure)

      chain = structure.get_chain(chain_code)
      if not chain:
        chain = Chain(structure, chain_code)

      # create new residue  
      residue = chain.get_residue(seqId)
      if not residue:
        residue = Residue(chain, seqId, resName)
      
     # missing element -> guess using first character of the atom name
      if not element:
        element = name[0]

      coords = (x,y,z)
      atom = Atom(residue, atomName, coords, element)
    
  fileObject.close()

  return structures

In [2712]:
# 3D group of molecules
class Structure:
    
  # creates instance of self  
  def __init__(self, name, conformation=0, pdbId=None):
    # exception for blank name
    if not name:
      raise Exception('Name is blank')
    self._name = name
    self.conformation = conformation
    self.pdbId = pdbId
    # links structure to children (chains)
    self.chains = []

  # chains delete when structure disappears
  def delete(self):
    for chain in self.chains:
      chain.delete()
    
  # find chain with matching atribute (loops through list of children)
  def get_chain(self, code):
    for chain in self.chains:
      if chain.code == code:
        return chain

    return None

  # gets a list of pdbIds
  def get_pdb_ids(self):
    return list(self.pdbIds)

  # gets molecular mass of structure
  def get_mass(self):
    if hasattr(self, 'mass'):
      return self.mass

    mass = 0
    for chain in self.chains:
      mass += chain.get_mass()
    self.mass = mass # cache for next time
    return mass

  # name of molecule as string
  def get_name(self):
    return self._name

  def set_name(self, name):
    if not name:
      raise Exception('Name is blank')
    self._name = name

  name = property(get_name, set_name)

In [2713]:
# child of Structure
# each molecule defined by chain class

class Chain:
  allowed_mol_types = ('protein', 'DNA', 'RNA')

  
  def __init__(self, structure, code, mol_type='protein'):
    if not code:
      raise Exception('Error. Code is blank.')
    
    if mol_type not in self.allowed_mol_types:
      raise Exception('Incorrect molecule type' %
                      (mol_type, self.allowed_mol_types))

    # checks for unique code
    chain = structure.get_chain(code)
    if chain:
      raise Exception('Code already used.' % code)

    # add chain to structure.chains list
    self.structure = structure
    self.code = code
    self._mol_type = mol_type
    self.resDict = {}
    self.residues = []
    structure.chains.append(self)

  # removes from structure list of children
  def delete(self):
    for residue in self.residues:
      residue.delete()
    self.structure.chains.remove(self)

  # returns child residue with given seqId
  def get_residue(self, seqId):
    return self.resDict.get(seqId)

  # returns atoms of all residues
  def get_atoms(self):
    atoms = []
    for residue in self.residues:
      atoms.extend(residue.atoms)
    return atoms
    
  # returns molecule type  
  def get_mol_type(self):
    return self._mol_type

  mol_type = property(get_mol_type)

In [2714]:
# child of Chain. seqId links to Chain
class Residue:
    
  def __init__(self, chain, seqId, code=None):
    if not seqId:
      raise Exception('seqId is blank')
    residue = chain.get_residue(seqId)
    if residue:
      raise Exception('seqId="%s" already used' % seqId)

    # add residues to chain.residues list
    self.chain = chain
    self.seqId = seqId
    self.code = code
    self.atomDict = {}
    self.atoms = []
    chain.resDict[seqId] = self
    chain.residues.append(self)

  # removes from chain
  def delete(self):
    for atom in self.atoms:
      atom.delete()
    del self.chain.resDict[self.seqId]
    self.chain.residues.remove(self)

  def get_atom(self, name):
    return self.atomDict.get(name)

In [2715]:
# child of Residue. 
class Atom:
  def __init__(self, residue, name, coords, element):
    if not name:
      raise Exception('Name is blank')
    atom = residue.get_atom(name)
    if atom:
      raise Exception('name="%s" already used' % name)
    if len(coords) != 3:
      raise Exception('Coordinates must contain three values')
 
    self.residue = residue
    self.name = name
    self.coords = array(coords)
    self.element = element

    residue.atomDict[name] = self
    residue.atoms.append(self)

  def delete(self):
    del self.residue.atomDict[self.name]
    self.residue.atoms.remove(self)

In [2716]:
# calculates center of mass, zero point
def center_coords(coords, weights):

  wCoords = coords.transpose() * weights

  # sum xyz total weights
  xyzTotals = wCoords.sum(axis=1)

  # average weight -> center
  center = xyzTotals/sum(weights)

  # remove previous center
  coords -= center
  
  return coords, center  

In [2717]:
# calculates best rotation superimposing two arrays
def align_coords(coordsA, coordsB, weights=None):

  # no weights, new weights created with series of one's
  n = len(coordsA)
  if weights is None:
    weights = ones(n)

  # initial matrix -> weighted dot product of 1 coordinate array 
  # and transpose of 2nd array
  rMat = dot(coordsB.transpose()*weights, coordsA)

  # singular value decomposition -> rotatio matrix, 
  # linear scaling factors, opposing rotation matrix
  rMat1, scales, rMat2 = linalg.svd(rMat)

  # checks for mirror image
  sign = linalg.det(rMat1) * linalg.det(rMat2)

  if sign < 0:
    rMat1[:,2] *= -1
  
  # optimized rotation matrix from 2 matrices from linalg.svd
  rotation = dot(rMat1, rMat2)
    
  # applies the transformation  
  coordsB = dot(coordsB, rotation)
  
  return rotation, coordsB

In [2718]:
# calculates root mean square deviation
# input reference coordinates, list of arrays to compare, list of weights

def calc_rmsds(ref_coord, all_coor, weights):

  rmsds = []
  # sum all weights
  totalWeight = sum(weights)
  # array of zeros same size as coordinate arrays
  totalSquares = zeros(ref_coord.shape)
  
  # loop through list of arrays and compare to reference
  for coords in all_coor:
    delta = coords-ref_coord
    squares = delta * delta
    totalSquares += squares
    # sum of the square values along the spatial axis
    sumSquares = weights*squares.sum(axis=1)
    # average atomic square deviation
    rmsds.append( sqrt(sum(sumSquares)/totalWeight) )
  
  # each atom’s rmsd over whole set of conformations
  nStruct = len(all_coor)
  atom_rmsds = sqrt(totalSquares.sum(axis=1)/nStruct)
  
  return rmsds, atom_rmsds

In [2719]:
# superimposes all conformations of a structure ensemble
def superimpose_coords(all_coor, weights, threshold=5.0):

  nStruct = len(all_coor)

  # reference set to first array
  ref_coord = all_coor[0]
  # create array of zeros of same size
  mean_coord = zeros(ref_coord.shape)
  rotations = []

  # loop through all arrays and align to reference
  for index, coords in enumerate(all_coor):
    # if index is zero an identity matrix is created
    if index == 0:
      rotation = identity(3)
    else:  
      rotation, coords = align_coords(ref_coord, coords, weights)
      all_coor[index] = coords # Update to aligned
      
    # rotation matrix is added to rotations list
    rotations.append(rotation)
    # coordinate array added to mean_coords
    mean_coord += coords
  
  # average positions
  mean_coord /= nStruct
  
  # calculate RMSD values
  rmsds, atom_rmsds = calc_rmsds(mean_coord, all_coor, weights)  
  
  # finds smallest rmds value
  best_rmsd = min(rmsds)
  best_index = rmsds.index(best_rmsd)
  best_coords = all_coor[best_index]

  # rmds value is scaled by a threshold to assign sensitivity to structural variation
  weight_scale = atom_rmsds/threshold
  # atoms with small rmsd have largest weights
  # as variance increases the weight diminishes exponentially
  weights = (weights * (exp(-weight_scale*weight_scale)))
  
  # copies coordinate array for average coordinates
  mean_coord = best_coords.copy()
  
  # loop of all coordinates to compare to average
  for index, coords in enumerate(all_coor):
    if index != best_index:
      rotation, coords = align_coords(best_coords, coords, weights)
      rotations[index] = rotation
      all_coor[index] = coords
      mean_coord += coords
 
  # calculate coordinate average
  mean_coord /= nStruct 

  # calculates rmsd of structures and individual atoms
  weights = ones(len(weights))
  rmsds, atom_rmsds = calc_rmsds(mean_coord, all_coor, weights)
  
  return all_coor, rmsds, atom_rmsds, rotations

In [2720]:
# extracts coordinate arrays, superimposes, updates original array
def super_impose(structures):

  weights = None
  all_coor = []

  # convert atom coordinates and atom list to NumPy array
  for structure in structures:
    atoms, coords = get_atom_coords(structure)
    
    if weights is None:
      weights = [atomic_numbers[atom.element] for atom in atoms]
      weights = array(weights)

    # center coordinates according to weight
    coords, center = center_coords(coords, weights)
    all_coor.append(coords)
  
  # superimpose arrays
  results = superimpose_coords(all_coor, weights)
  # output: rmsd for atom and structure and list of rotaations
  all_coor, rmsds, atom_rmsds, rotations = results  
  
  # update atom objects with new coordinates
  for i, structure in enumerate(structures):
    atoms, oldCoords = get_atom_coords(structure)
    
    for j, atom in enumerate(atoms):
      atom.coords = all_coor[i][j]

  return rmsds, atom_rmsds

In [2721]:
# converts 3 letter amino acids to 1 letter

def get_chain_sequence(chain):

  letters = []
  
  # loops through residues in input chain
  for residue in chain.residues:
    code =  residue.code
    # X for unknown amino acid (not in amino_acids dictionary)
    letter = amino_acids.get(code, 'X')
    letters.append(letter)
    
  seq = ''.join(letters)

  return seq

In [2722]:
# pairwise sequence alignment
# imput: seqA and seqB; substitution matrix, gap penalties

def sequence_align(seqA, seqB, simMatrix=DNA_1, insert=8, extend=4):

  # define comparison grid: +1 larger then length of sequences
  numI = len(seqA) + 1
  numJ = len(seqB) + 1

  # create route matrix
  scoreMatrix = [[0] * numJ for x in range(numI)]
  routeMatrix = [[0] * numJ for x in range(numI)]
  
  # edges of matrix at zero
  for i in range(1, numI):
    routeMatrix[i][0] = 1
  
  for j in range(1, numJ):
    routeMatrix[0][j] = 2
  
  # fill in remaining matrix
  for i in range(1, numI):
    for j in range(1, numJ):
    
      # insertion penalty unless previous has gap then extension penalty
      penalty1 = insert
      penalty2 = insert
      
      if routeMatrix[i-1][j] == 1:
        penalty1 = extend
        
      elif routeMatrix[i][j-1] == 2:
        penalty2 = extend

      # similarity score
      similarity = simMatrix[ seqA[i-1] ][ seqB[j-1] ]
        
      # list of 3 possible paths
      paths = [scoreMatrix[i-1][j-1] + similarity, # Route 0
               scoreMatrix[i-1][j] - penalty1, # Route 1
               scoreMatrix[i][j-1] - penalty2] # Route 2                     
      
      # maximum value is best score
      best = max(paths)
      route = paths.index(best)           

      scoreMatrix[i][j] = best
      routeMatrix[i][j] = route

  # add in penalties
  alignA = []
  alignB = []
  
  i = numI-1
  j = numJ-1
  score = scoreMatrix[i][j]

    
  while i > 0 or j > 0:
    route = routeMatrix[i][j]    

    # best score from comparing 2 residues
    if route == 0: 
      alignA.append( seqA[i-1] )
      alignB.append( seqB[j-1] )
      i -= 1
      j -= 1

    # best score from inserting a gap
    elif route == 1: # Gap in seqB
      alignA.append( seqA[i-1] )
      alignB.append( '-' )
      i -= 1 
    
    # best score from inserting a gap
    elif route == 2: # Gap in seqA
      alignA.append( '-' )
      alignB.append( seqB[j-1] ) 
      j -= 1
  
  # correct order of lists
  alignA.reverse()
  alignB.reverse()
  alignA = ''.join(alignA)
  alignB = ''.join(alignB)

  return score, alignA, alignB 

In [2723]:
# function for sequence alignment of corresponding residues
def backbone_align(chainA, chainB, 
                              atomNames=set(['CA','C','N']),
                              simMatrix=BLOSUM62):

  # input chains and list of residue objects
  structureA = chainA.structure
  structureB = chainB.structure
  residuesA = chainA.residues
  residuesB = chainB.residues

  # get 1 letter sequence strings from Chain
  seqA = get_chain_sequence(chainA)
  seqB = get_chain_sequence(chainB)

  score, alignA, alignB = sequence_align(seqA, seqB, simMatrix)

  # list to hold aligned residues
  pairedPosA = []
  pairedPosB = []  
  posA = 0
  posB = 0
    

  # loop to find residues in alignment string from sequence_align
  # gaps increase counter +1
  for i in range(len(alignA)):
 
    if alignA[i] == '-':
      posB += 1
   
    elif alignB[i] == '-':
      posA += 1
    
    else:
      pairedPosA.append(posA)
      pairedPosB.append(posB)
      
      posA += 1
      posB += 1

  # uses positions to place seqId in list
  filterIdsA = [residuesA[p].seqId for p in pairedPosA]
  filterIdsB = [residuesB[p].seqId for p in pairedPosB]


  # filter to contain backbone structures
  backboneStrucA = filter_structure(structureA, [chainA.code], 
                                      filterIdsA, atomNames)
  backboneStrucB = filter_structure(structureB, [chainB.code], 
                                      filterIdsB, atomNames)
  

  # list of atoms and coordinate array
  atomsA, coordsA = get_atom_coords(backboneStrucA)
  atomsB, coordsB = get_atom_coords(backboneStrucB)
  weights = ones(len(atomsA))


  # coordinates are centered
  coordsA, centerA = center_coords(coordsA, weights)
  coordsB, centerB = center_coords(coordsB, weights)

  # superiposes coordinates
  coords = [coordsA, coordsB]
  c, rmsds, atom_rmsds, rotations = superimpose_coords(coords, weights)

  # rotations and center transforms original structures
  rotated_structure(structureA, rotations[0], -centerA)  
  rotated_structure(structureB, rotations[1], -centerB)  
    
  return rmsds, atom_rmsds

In [2724]:
# generates rotation matrix
# uses axis direction and angle

def get_rotation_matrix(axis, angle):
  vLen = math.sqrt(sum([xyz*xyz for xyz in axis]))
  x, y, z = [xyz/vLen for xyz in axis]
  c = math.cos(angle)
  d = 1-c
  s = math.sin(angle)

  R = [[c+d*x*x,   d*x*y-s*z, d*x*z+s*y],
       [d*y*x+s*z, c+d*y*y,   d*y*z-s*x],
       [d*z*x-s*y, d*z*y+s*x, c+d*z*z  ]]
    
  return R

In [2725]:
# rotates structure while keeping relative position of coordinates
def rotate_structure_NumPy(structure, axis=array([2,1,1]), angle=0):

  # convert to array
  rotated_matrix = array(get_rotation_matrix(axis, angle))

  # get atom array with coordinates
  atoms, coords = get_atom_coords(structure)
  
  coords = dot(coords, rotated_matrix.T)
  
  for index, atom in enumerate(atoms):
    atom.coords = list(coords[index])

In [2726]:
# transforms coordinates
# uses identity matrix with size 3 (to match coordinates)
def rotated_structure(structure, transform=identity(3),
                             translate=(0,0,0)):

  # get atom array with coordinates
  atoms, coords = get_atom_coords(structure)

  # apply transformation
  coords = dot(coords, transform.T)
  
  # apply translation
  coords = coords + translate

  for index, atom in enumerate(atoms):
    atom.coords = coords[index]

In [2727]:
# includes only required atoms
def filter_structure(structure, chain_codes=None,
                       residueIds=None, atomNames=None):

  name = structure.name + '_filter'

  # checks if chain, residue, and atom names are defined
  if chain_codes:
    name += ' ' + ','.join(chain_codes)
    chain_codes = set(chain_codes)
  
  if residueIds:
    name += ' ' + ','.join([str(x) for x in residueIds])
    residueIds = set(residueIds)

  if atomNames:
    name += ' ' + ','.join(atomNames)
    atomNames = set(atomNames)

    
  # create copy of structure
  conf = structure.conformation
  pdbId = structure.pdbId
  filterStruc = Structure(name=name, conformation=conf, pdbId=pdbId)

    
  # loop for selecting chains, residues, and atoms
  for chain in structure.chains:
    if chain_codes and (chain.code not in chain_codes):
      continue
  
    includeResidues = []
    
    for residue in chain.residues:
      if residueIds and (residue.seqId not in residueIds):
        continue

      includeAtoms = []

      for atom in residue.atoms:
        if atomNames and (atom.name not in atomNames):
          continue

        includeAtoms.append(atom)

      if includeAtoms:
        includeResidues.append( (residue, includeAtoms) )

        
    # creates new chain
    if includeResidues:
        
      filterChain = Chain(filterStruc, chain.code, chain.mol_type)
      
      for residue, atoms in includeResidues:
        filterResidue = Residue(filterChain, residue.seqId,
                                residue.code)
    
        for atom in atoms:
          coords = array(atom.coords)
          Atom(filterResidue, atom.name, coords, atom.element)
  

  return filterStruc

In [2728]:
def get_atom_coords(structure):

  coords = []
  atoms = []
  
  for chain in structure.chains:
    for residue in chain.residues:
      for atom in residue.atoms:
        coords.append(atom.coords)
        atoms.append(atom)

  return atoms, array(coords) 
  

In [2729]:
if __name__ == '__main__':

  print('\nTest Root-Mean_Square-Deviation \nStructure A and Rotated Structure A:')
 
  strucA = get_structures(download_PDB('5UDO'))[0]
  strucB = get_structures(download_PDB('5UDO'))[0]
  # rotate file to get strucB
  rotate_structure_NumPy(strucA, (1,0,0), pi/3)
  rmsds, atom_rmsds = super_impose([strucA, strucB])
  print(rmsds) 
 

  print('\nSuperimpose Two Homologous Structures')
  # input two homologous structures
  struc1 = get_structures(download_PDB('5UDO'))[0]
  struc2 = get_structures(download_PDB('5U96'))[0]
  chain1 = struc1.get_chain('A')
  chain2 = struc2.get_chain('A')
  rmsds, atom_rmsds = backbone_align(chain1, chain2)

  print(rmsds)


Test Root-Mean_Square-Deviation 
Structure A and Rotated Structure A:
[3.5437596409757814e-14, 3.5435795315108795e-14]

Superimpose Two Homologous Structures
[0.5572256141745706, 0.5572256141745703]
